In [ ]:
import json

import os
import sys

import torch
from pytorch_slim_cnn.slimnet import SlimNet
from torchvision import transforms
from PIL import Image
import numpy as np

torch.cuda.set_device(2)

In [ ]:
class FFHQ(object):
    def __init__(self, root, transform=None):
        self.root = os.path.expanduser(root)
        self.transform = transform
        from torchvision.datasets.folder import default_loader
        self.loader = default_loader
        self.info = None

    def __getitem__(self, key):
        if self.info is None:
            with open(os.path.join(self.root, 'ffhq-dataset-v2.json'), 'r') as f:
                self.info = json.load(f)
                
        path = os.path.join(self.root, self.info[str(key)]['image']['file_path'])
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        return key, sample
            
    def __len__(self):
        return 70000

In [ ]:
transform = transforms.Compose([
                              transforms.Resize((178,218)),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])
ffhq = FFHQ('/data/vision/torralba/datasets/ffhq', transform=transform)

In [ ]:
loader = torch.utils.data.DataLoader(ffhq, num_workers=20, batch_size=512, pin_memory=True)

In [ ]:
device = torch.device('cuda')

In [ ]:

labels = np.array(['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
       'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
       'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
       'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young'])

In [ ]:
model = SlimNet.load_pretrained('./pytorch_slim_cnn/models/celeba_20.pth').to(device).eval()


In [ ]:
from tqdm.auto import tqdm

In [ ]:
with open('/data/vision/torralba/datasets/ffhq/ffhq-dataset-v2.json', 'r') as f:
    info = json.load(f)

In [ ]:
with torch.no_grad():
    for indices, x in tqdm(loader):
        logits = model(x.to(device))
        sigmoid_logits = torch.sigmoid(logits)
        predictions = (sigmoid_logits > 0.5).squeeze().cpu().numpy().astype(bool)
        for idx, p in zip(indices, predictions):
            info[str(idx.item())]['image']['labels'] = labels[p].tolist()

In [ ]:
!ls

In [ ]:
!pwd

In [ ]:
with open('ffhq_json/ffhq_labeled.json', 'w') as f:
    json.dump(info, f)

In [ ]:
!mv ffhq_labeled.json ffhq_json

In [ ]:
info['23']['image']

In [ ]:
smile_info = {k: v for k, v in info.items() if 'Smiling' in v['image']['labels']}

In [ ]:
Image.open('/data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/clean/clean_0.png').size

In [ ]:
!ls /data/vision/torralba/ganprojects/placesgan/tracer/utils/samples/clean/

In [ ]:
with open('ffhq_json/ffhq_labeled_smiling.json', 'w') as f:
    json.dump(smile_info, f)

In [ ]:


PATH_TO_IMAGE = '/data/vision/torralba/datasets/ffhq/images1024x1024/53000/53990.png'

# GPU isn't necessary but could definitly speed up, swap the comments to use best hardware available
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

# Make tensor and normalize, add pseudo batch dimension and move to configured device
with open(PATH_TO_IMAGE, 'rb') as f:
    x = transform(Image.open(f)).unsqueeze(0).to(device)

model = SlimNet.load_pretrained('./pytorch_slim_cnn/models/celeba_20.pth').to(device)

with torch.no_grad():
            model.eval()
            logits = model(x)
            sigmoid_logits = torch.sigmoid(logits)
            predictions = (sigmoid_logits > 0.5).squeeze().numpy()

print(labels[predictions.astype(bool)])

In [ ]:
PATH_TO_IMAGE = '/data/vision/torralba/datasets/ffhq/images1024x1024/53000/53990.png'

In [ ]:
Image.open(PATH_TO_IMAGE)